# Segmentation 

## Prepare data
merge channels

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
from data import DivideData
import random

In [4]:
def rx_values(i_values, q_values):
    a1i = np.array([np.array(row) for row in i_values])
    a1q = np.array([np.array(row) for row in q_values])
    
    n = np.array(range(256)) / 256
    f_c = 24_000_000_000  # 24 GHz
    v = 2.0 * np.pi * f_c * n
    
    value_rx = a1i * np.cos(v) + a1q + np.sin(v)   
    return value_rx, a1i, a1q

In [5]:
dt = pd.read_parquet('data/parquet_samples/13_06_22/radar_samples_192.168.67.112_412.parquet')

split by participants first!

In [6]:
def prepare_arr(dt):
    rx1_a_i = dt['rx1_freq_a_channel_i_data']
    rx1_a_q = dt['rx1_freq_a_channel_q_data']
    rx2_a_i = dt['rx2_freq_a_channel_i_data'] 
    rx2_a_q = dt['rx2_freq_a_channel_q_data']
    rx1_b_i = dt['rx1_freq_b_channel_i_data']
    rx1_b_q = dt['rx1_freq_b_channel_q_data']

    # merge!
    a1_rx, a1_i_values, a1_q_values = rx_values(i_values=rx1_a_i,  q_values=rx1_a_q)
    a2_rx, a2_i_values, a2_q_values = rx_values(i_values=rx2_a_i,  q_values=rx2_a_q)
    b1_rx, b1_i_values, b1_q_values = rx_values(i_values=rx1_b_i,  q_values=rx1_b_q)

    #With the Merge function, now the channel of i and q are combine together, but it is a 2D array, need to reshape it to 1D
    a1_rx_1d = a1_rx.reshape(-1)
    a2_rx_1d = a2_rx.reshape(-1)
    b1_rx_1d = b1_rx.reshape(-1)

    #convert all three channels together and use information gain for segmentation
    merged_arrs = np.vstack((a1_rx_1d, a2_rx_1d, b1_rx_1d))

    return merged_arrs

In [7]:
def igts_usage(arr, k, step, par_num, count):
    # IGTS
    TD_TT, IG_arr, knee = TopDown(arr, k, step=step)
    print(f'knee={knee}')
    #k = knee
    #TD_TT, IG_arr, knee = TopDown(arr, k, step=step)
    print(TD_TT)
    #plot_segments(arr, TD_TT, f'/home/Shared/xinyi/blob1/thesis/figure/IGTS/IGTS_igcul_k={k}_s={step}_{par_num}_{count}')
    return TD_TT

In [10]:
divide_data = DivideData(dt, '/home/Shared/xinyi/blob1/thesis')
participants, num_par = divide_data.divide_participants()
participants['participant_1']

In [24]:
participant = participants['participant_1']
tasks = pd.read_csv('/home/Shared/xinyi/blob1/thesis/data/task.csv')
task_index = {}
for i in range(15):
    T = participant[participant['task_uuid'] == tasks['UUID'][i]].index
    task_index[f'T_{i}'] = T

In [26]:
task_index

In [21]:
important_task = ['T_0', 'T_6', 'T_7', 'T_11']
for key, var in enumerate(task_index):
    if var in important_task:
        

In [9]:
for par in participants:
    participant = participants[par]
    par_dt = participant.iloc[:,:6]
    print(par_dt)

In [90]:
for par in participants:
    participant = participants[par]
    par_dt = participant.iloc[:,:6]
    merged_arrs = prepare_arr(par_dt)
    
    #split the arrays
    merged_arrs_split = np.array_split(merged_arrs, 10, axis=1)
    
    count = 1
    for split_arr in merged_arrs_split:
        
        igts_usage(split_arr, k=10, step=10, par_num=par, count=count)
    

In [93]:
for par in participants:
    participant = participants[par]
    par_dt = participant.iloc[:,:6]
    merged_arrs = prepare_arr(par_dt)
    
    #split the arrays
    merged_arrs_split = np.array_split(merged_arrs, 10, axis=1)
    
    count = 1
    for split_arr in merged_arrs_split:
        segments = igts_usage(split_arr, k=10, step=10, par_num=par, count=count)
        
        for i in range(data.shape[0]):
            plt.plot(data[i], label=f'Channel {i+1}', alpha=0.7)

        for seg in segments:
            random_color = [random.random() for _ in range(3)]
            plt.axvline(x=seg, color=random_color, linestyle='--')

        plt.title('Time Series Segmentation')
        plt.xlabel('Time')
        plt.ylabel('Value')
        plt.legend()
        plt.show()